In [ ]:
#过滤掉签到记录少于10次的用户，被签到次数少于10次的地点，并对用户和位置重新编号
import pandas as pd

data = pd.read_csv('dataset_TKY.txt', encoding="unicode_escape", names=['use_ID', 'loc_ID', 'loc_cat_ID', 'loc_cat_name', 'latitude', 'longitude', 'time_offset', 'time'], sep='\t')
data_exact = data[['use_ID', 'loc_ID', 'latitude', 'longitude', 'time']]
data_exact['time'] = pd.to_datetime(data_exact['time'])
data_exact['time'] = data_exact['time'] + pd.Timedelta(minutes=540)#将time更改为本地时间

In [4]:
#过滤数据
def filter(data):
    dm = data[['use_ID', 'loc_ID']]
    user_cnt = dm.groupby(by=['use_ID'], as_index=False).count()
    user_list = list(user_cnt[user_cnt['loc_ID'] >= 10]['use_ID'])#过滤掉签到次数少于10次的用户
    ds = data[data['use_ID'].isin(user_list)]
    loc_cnt = ds.groupby(by=['loc_ID'], as_index=False).count()
    loc_list = list(loc_cnt[loc_cnt['use_ID'] >= 10]['loc_ID'])#过滤掉签到人数少于10人次的位置
    data_filtered = ds[ds['loc_ID'].isin(loc_list)]
    return data_filtered
data_filtered = filter(data_exact)
user_cnt = data_filtered.groupby(by=['use_ID'], as_index=False).count()
control = True
while control:
    if (any(user_cnt['loc_ID'] < 10)):
        data_filtered = filter(data_filtered)
        user_cnt = data_filtered.groupby(by=['use_ID'], as_index=False).count()
    else:
        control = False
data_filtered.sort_values(by='time', inplace=True)#按时间排序
data_filtered.index = range(len(data_filtered))
data_filtered.to_csv('FourSquare_TKY.csv', sep=',', index=False, header=True)

In [5]:
loc_n = data_exact['loc_ID'].drop_duplicates()

In [6]:
#数据集划分：按时序，8:1:1（细节：在分割点处，保证一天的完整轨迹）
import pandas as pd
data = pd.read_csv('FourSquare_TKY.csv', names=['use_ID', 'loc_ID', 'latitude', 'longitude', 'time'], sep=',', header=0)
data['time'] = pd.to_datetime(data['time'])
train = data.head(int(0.8*447570+122))#+122：将分割时间点所在的一天的签到记录归纳到训练集或验证集
tmp = data.head(int(0.9*447570+409))
valid = tmp.tail(403222-358178)
test = data.tail(447570-403222)

In [ ]:
#将在训练集中没有而在验证集中出现的用户ID或者位置ID的相关签到记录排除
train_user = set(train['use_ID'])
val_user = set(valid['use_ID'])
test_user = set(test['use_ID'])
train_loc = set(train['loc_ID'])
val_loc = set(valid['loc_ID'])
test_loc = set(test['loc_ID'])
val_g = valid.groupby(by='use_ID')
val_train_user = val_user - train_user
for use_id, valg in val_g:
    if use_id in val_train_user:
        valid.drop(valg.index, inplace=True)
val_g = valid.groupby(by='loc_ID')
val_train_loc = val_loc - train_loc
for loc_id, valg in val_g:
    if loc_id in val_train_loc:
        valid.drop(valg.index, inplace=True)
        
#将在训练集中没有而在测试集中出现的用户ID或者位置ID的相关签到记录排除
test_g = test.groupby(by='use_ID')
test_train_user = test_user - train_user
for use_id, valg in test_g:
    if use_id in test_train_user:
        test.drop(valg.index, inplace=True)
test_g = test.groupby(by='loc_ID')
test_train_loc = test_loc - train_loc
for loc_id, valg in test_g:
    if loc_id in test_train_loc:
        test.drop(valg.index, inplace=True)
train.to_csv('train.csv', sep=',', index=False, header=True)
valid.to_csv('valid.csv', sep=',', index=False, header=True)
test.to_csv('test.csv', sep=',', index=False, header=True)

In [129]:
#处理后，验证集和测试集中出现的用户和位置均在训练集中出现过
#可以对训练集中的用户和位置重新编号
data = pd.read_csv('train.csv', names=['use_ID', 'loc_ID', 'latitude', 'longitude', 'time'], sep=',', header=0)
user = list(set(data['use_ID']))
user_id = {}
for i, user in enumerate(user):
    user_id[user] = i  #用户新编号
user_id = pd.DataFrame(list(user_id.items()))
user_id.columns = ['old_id', 'new_id']
user_id.to_csv('user_id.csv', sep=',', index=False, header=True)

In [ ]:
#存在位置ID一致，但是经纬度坐标不一致的情况，由于要对地理空间影响显式建模，因此需要做一定处理，暂采用均值处理
import pandas as pd
data = pd.read_csv('train.csv', names=['use_ID', 'loc_ID', 'latitude', 'longitude', 'time'], sep=',', header=0)
loc = data[['loc_ID', 'latitude', 'longitude']].drop_duplicates()
loc_g = loc.groupby(by=['loc_ID'], as_index=False).count()
multi_loc = loc_g[loc_g['latitude']>=2]#提取loc_ID一致，但是经纬度不一致的数据
multi_loc.index = range(len(multi_loc))
multi_loc_item = loc[loc['loc_ID'].isin(multi_loc['loc_ID'])]
loc_only_coor = pd.DataFrame()
for i in range(len(multi_loc)):
    a = multi_loc_item[multi_loc_item['loc_ID'].isin([multi_loc['loc_ID'][i]])].mean()#经纬度各自求均值
    loc_only_coor = pd.concat([loc_only_coor,a],axis=1)
loc_only_coor = loc_only_coor.T
loc_only_coor.index = range(len(loc_only_coor))
loc_only_coor['loc_ID'] = multi_loc['loc_ID']
loc_only = loc_only_coor[['loc_ID', 'latitude', 'longitude']]
loc_del_multi = loc[~loc['loc_ID'].isin(multi_loc['loc_ID'])]
loc_global = pd.concat([loc_del_multi, loc_only])
loc_global.index = range(len(loc_global))
loc_global['loc_new_ID'] = loc_global.index #赋予新ID
loc_global.to_csv('loc.csv', sep=',', index=False, header=True)

In [ ]:
#将训练集、验证集、测试集中的用户ID和位置ID替换成新ID
import pandas as pd
data = pd.read_csv('test.csv', names=['use_ID', 'loc_ID', 'latitude', 'longitude', 'time'], sep=',', header=0)
loc = pd.read_csv('loc.csv', names=['loc_ID', 'latitude', 'longitude', 'loc_new_ID'], sep=',', header=0)
user = pd.read_csv('user_id.csv', names=['old_id', 'new_id'], sep=',', header=0)
user_id = {}
loc_id = {}
for i in range(len(user)):
    user_id[user['old_id'][i]] = user['new_id'][i]
for i in range(len(loc)):
    loc_id[loc['loc_ID'][i]] = loc['loc_new_ID'][i]
data['user_new_ID'] = -1
data['loc_new_ID'] = -1
for i in range(len(data)):
    data['user_new_ID'][i] = user_id[data['use_ID'][i]]  
    data['loc_new_ID'][i] = loc_id[data['loc_ID'][i]] 
train_data = data[['user_new_ID', 'loc_new_ID', 'time']]
train_data.to_csv('TKY_test.csv', sep=',', index=False, header=True)

In [ ]:
#将训练集、验证集、测试集签到数据按用户，连续两次签到间隔超过24小时，拆分为不同的轨迹
import pandas as pd
import json
data = pd.read_csv('TKY_train.csv', names=['user_new_ID', 'loc_new_ID', 'time'], sep=',', header=0)
data.sort_values(['user_new_ID','time'], ascending=[True, True], inplace=True)
data.index = range(len(data))
#生成每条签到记录的轨迹号
data['time'] = pd.to_datetime(data['time'])
data_g = data.groupby(by='user_new_ID')
data_traj = pd.DataFrame()
for user, datag in data_g:
    traj = 0
    datag['traj_num'] = -1
    datag.index = range(len(datag))
    f = 0
    for i in range(len(datag)-1):
        deltatime = datag['time'][i+1] - datag['time'][i]
        if deltatime.days >= 1:
            datag['traj_num'][f:i+1] = traj
            f = i + 1
            traj += 1
    datag['traj_num'][f:] = traj
    data_traj = pd.concat((data_traj, datag))
data_traj.index = range(len(data_traj))
#生成每条签到记录所在的星期几名字
week = [data_traj['time'][i].isoweekday() for i in range(len(data_traj))]
data_traj['day_of_week'] = week
#生成每条签到记录的时间片
tm = data_traj['time']
tms = list(map(lambda x: x.hour * 2 + 2 if x.minute >=30 else x.hour * 2 + 1, tm))
data_traj['timeslot'] = pd.Series(tms)
data_traj.to_csv('train_traj.csv', sep=',', index=False, header=True)

In [ ]:
#提取轨迹序列
import numpy as np
data_traj_g = data_traj.groupby(by=['user_new_ID', 'traj_num'])
for i, traj in data_traj_g:
    d_t = traj[['loc_new_ID', 'timeslot', 'day_of_week']]
    if len(d_t) > 1:
       d_t = np.array(d_t)

In [6]:
data_traj_g = data_traj.groupby(by=['user_new_ID'])
traj_all = []
# traj_all = dict()
for i, u_traj in data_traj_g:
    u_traj_g = u_traj.groupby(by=['traj_num'])
    traj_src = dict()
    traj_pred = dict()
    for n, d_traj in u_traj_g:
        d_t = d_traj[['loc_new_ID', 'timeslot', 'day_of_week']]
        if len(d_t) > 1: #排除只有一个轨迹点的轨迹
            traj_points = [(d_t.iloc[j][0], d_t.iloc[j][1], d_t.iloc[j][2]) for j in range(len(d_t))]
        # if len(traj_points) > 1:
            traj_src[n] = traj_points[:-1]
            traj_pred[n] = traj_points[-1]
    # traj_all[i] = {'forward': traj_src, 'pred': traj_pred}
    if len(traj_pred) > 0:
        traj_all.append({i: {'forward': traj_src, 'pred': traj_pred}})

In [ ]:
#提取轨迹内的转移边，构造全局转移图 
import pandas as pd
data = pd.read_csv('TKY_train.csv', names=['user_new_ID', 'loc_new_ID', 'time'], sep=',', header=0)
data.sort_values(['user_new_ID','time'], ascending=[True, True], inplace=True)
data.index = range(len(data))
data['time'] = pd.to_datetime(data['time'])
data_g = data.groupby(by='user_new_ID')
data_traj = pd.DataFrame()
#提取用户的各条轨迹
for user, datag in data_g:
    traj = 0
    datag['traj_num'] = -1
    datag.index = range(len(datag))
    f = 0
    for i in range(len(datag)-1):
        deltatime = datag['time'][i+1] - datag['time'][i]
        if deltatime.days >= 1:
            datag['traj_num'][f:i+1] = traj
            f = i + 1
            traj += 1
    datag['traj_num'][f:] = traj
    data_traj = pd.concat((data_traj, datag))
data_traj.index = range(len(data_traj))

#提取轨迹内的转移边：共274667条转移连边（训练集）
data_traj_g = data_traj.groupby(by=['user_new_ID', 'traj_num'])
edge_trans = []
for i, traj in data_traj_g:
    traj.index = range(len(traj))
    trans_e = [(traj['loc_new_ID'][j], traj['loc_new_ID'][j+1]) for j in range(len(traj) - 1)]
    edge_trans += trans_e

In [2]:
#统计转移边及其频次 有向边：src->dst：频次
from collections import Counter
import json
edge_trans_cnt = Counter(edge_trans)#最大频次为623
global_tran_e = dict(edge_trans_cnt)
global_tran_edge = pd.DataFrame(global_tran_e.keys())
global_tran_edge.columns = ['src', 'dst']
global_tran_edge['freq'] = global_tran_e.values()
gtedge_g = global_tran_edge.groupby(by='dst')
tran_edge = pd.DataFrame()
for dst, e in gtedge_g:
    e.index = range(len(e))
    total_freq = e['freq'].sum()
    e['weight'] = e['freq'] / total_freq
    tran_edge = pd.concat((tran_edge, e))
    tran_edge.to_csv('tran_edge.csv', sep=',', index=False, header=True)

In [1]:
#构建位置的地理空间连边
import pandas as pd
import transbigdata as tbd
import geohash as gh
from haversine import haversine
data = pd.read_csv('loc.csv', names=['loc_ID', 'latitude', 'longitude', 'loc_new_ID'], sep=',', header=0)
loc = data[['loc_new_ID', 'latitude', 'longitude']]
geohash = tbd.geohash_encode(loc['longitude'], loc['latitude'], precision=5)#precision:5(4.89*4.89km),6(1.22*0.61km)
geo_edge = []
for i in range(len(loc)):
    neigh = gh.neighbors(geohash[i])
    for k in neigh:
        loc_geo = geohash[geohash.values == neigh[k]]
        if len(loc_geo) == 0:
            continue
        else:
            # for j in loc_geo.index:
            #     dist = haversine((loc['latitude'][i], loc['longitude'][i]), (loc['latitude'][j], loc['longitude'][j]), unit='m')
            #     if dist <= 1000:#超参：距离小于1000m的两个位置建立连边
            #         geo_edge +=[(i, j)]
            geo_edge += [(i, j) for j in loc_geo.index if haversine((loc['latitude'][i], loc['longitude'][i]), (loc['latitude'][j], loc['longitude'][j]), unit='m')<=1000]
geo_edge = pd.DataFrame(geo_edge)
geo_edge.columns = ['src', 'dst']

In [3]:
geo_edge.to_csv('geo_edge.csv', sep=',', index=False, header=True)

In [1]:
import pandas as pd
import numpy as np
import pickle
traj_data = pd.read_csv('train_traj.csv', names=['user_new_ID','loc_new_ID','time','traj_num','day_of_week','timeslot'], sep=',', header=0)
data_traj_g = traj_data.groupby(by=['user_new_ID', 'traj_num'])
traj_f = []
traj_p = []
traj_user = []
traj_user_traj_n = []
for i, traj in data_traj_g:
    d_t = np.array(traj[['loc_new_ID', 'timeslot', 'day_of_week']])
    if len(d_t) > 1:#剔除只有一条签到记录的轨迹
        traj_tmp = [tuple(d_t[i]) for i in range(len(d_t))]
        traj_f.append(traj_tmp[:-1])
        # traj_p.append(traj_tmp[-1])#只提取最后一次
        traj_p.append(traj_tmp[1:])#从第2个签到记录开到最后
        traj_user.append(i[0])
        traj_user_traj_n.append(i[1])
file = open('train_forward.pickle','wb')
pickle.dump(traj_f,file)
file.close()
file = open('train_labels.pickle','wb')
pickle.dump(traj_p,file)
file.close()
file = open('train_user.pickle','wb')
pickle.dump(traj_user,file)
file.close()
# traj_p = pd.DataFrame(traj_p)
# traj_p.columns = ['loc_new_ID', 'timeslot', 'day_of_week']
# traj_user = pd.DataFrame(traj_user)
# traj_user.columns = ['user_new_ID']
# traj_pred = pd.concat((traj_user, traj_p), 1)
# traj_pred.to_csv('valid_pred.csv', sep=',', index=False, header=True)

In [2]:
list_file = open('train_forward.pickle','rb')
list2 = pickle.load(list_file)